In [ ]:
from IPython.display import HTML
HTML(open('../style.css').read())

# Resolving Conflicts Using *Precedence Declarations*

This file shows how *shift/reduce* and *reduce/reduce* conflicts can be resolved using *operator precedence declarations*.
The following grammar is *ambiguous* because it does not specify the precedence of the arithmetical operators:
```
    expr : expr '+' expr
         | expr '-' expr
         | expr '*' expr
         | expr '/' expr
         | expr '^' expr
         | '(' expr ')'
         | NUMBER      
         ;
```
We will see how the use of precedence declarations can be used to resolve *shift/reduce-conflicts*.

## Specification of the Scanner

We implement a minimal scanner for arithmetic expressions.

In [ ]:
import ply.lex as lex

tokens = [ 'NUMBER' ]

def t_NUMBER(t):
    r'0|[1-9][0-9]*'
    t.value = int(t.value)
    return t

literals = ['+', '-', '*', '/', '^', '(', ')']

t_ignore  = ' \t'

def t_newline(t):
    r'\n+'
    t.lexer.lineno += t.value.count('\n')

def t_error(t):
    print(f"Illegal character '{t.value[0]}'")
    t.lexer.skip(1)

__file__ = 'main'

lexer = lex.lex()

## Specification of the Parser

In [ ]:
import ply.yacc as yacc

The *start variable* of our grammar is `expr`, but we don't have to specify that.  The default
start variable is the first variable that is defined.

The following *operator precedence declarations* declare that the operators `'+'`and `'-'` have a lower precedence than the operators `'*'`and `'/'`.  The operator `'^'` has the highest precedence.  Furthermore, the declarations specify that the operators  `'+'`, `'-'`, `'*'`, and `'/'` are *left associative*, while the operator `'^'` is declared as *right associative* using the keyword `right`.
Operators can also be defined as being *non-associative* using the keyword `nonassoc`.

In [ ]:
precedence = ( ('left', '+', '-') ,  # precedence 1
               ('left', '*', '/'),   # precedence 2
               ('right', '^')        # precedence 3
             )

In [ ]:
def p_expr_plus(p):
    "expr : expr '+' expr"
    p[0] = ('+', p[1], p[3])
    
def p_expr_minus(p):
    "expr : expr '-' expr"
    p[0] = ('-', p[1], p[3])
    
def p_expr_mult(p): 
    "expr : expr '*' expr"
    p[0] = ('*', p[1], p[3])
    
def p_expr_div(p): 
    "expr : expr '/' expr"
    p[0] = ('/', p[1], p[3])    

def p_expr_power(p): 
    "expr : expr '^' expr"
    p[0] = ('^', p[1], p[3])

def p_expr_paren(p): 
    "expr : '(' expr ')'"
    p[0] = p[2]
    
def p_expr_NUMBER(p):
    "expr : NUMBER"
    p[0] = p[1]

In [ ]:
def p_error(p):
    if p:
        print(f"Syntax error at character number {p.lexer.lexpos} at token '{p.value}' in line {p.lexer.lineno}.")
    else:
        print('Syntax error at end of input.')

Setting the optional argument `write_tables` to `False` <B style="color:red">is required</B> to prevent an obscure bug where the parser generator tries  to read an empty parse table.

In [ ]:
yacc.yacc(write_tables=False, debug=True);

As there are no warnings all conflicts have been resolved using the precedence declarations.
Let's look at the action table that is generated.  

In [ ]:
!ls -l

In [ ]:
!cat parser.out

In [ ]:
%run ../AST2Dot.ipynb

The function `test(s)` takes a string `s` as its argument an tries to parse this string.  If all goes well, an abstract syntax tree is returned.
If the string can't be parsed,  an error message is printed by the parser.

In [ ]:
def test(s):
    t = yacc.parse(s)
    d = tuple2dot(t)
    display(d)
    return t

In [ ]:
test('2^3^4*5+6-7/8^9')

In [ ]:
test('1+2*3^4')

In [ ]:
test('1 * 2 + (3^4)^5')